# Анализ функций разладки

## Инициализация

In [ ]:
from copy import deepcopy
import warnings

import numpy as np
import matplotlib.pyplot as plt
import rpy2.robjects as robjects
import rpy2.robjects.numpy2ri
from rpy2.robjects.packages import importr
import openpyxl
import pandas as pd
from utils.hmatr import Hmatr

from utils.modelling import modellingSeriesStatistics
from utils.modelling import fixSeriesStatistics

%load_ext autoreload
%load_ext rpy2.ipython
%autoreload 2

warnings.filterwarnings('ignore')
rpy2.robjects.numpy2ri.activate()
# utils = importr('utils')

# utils.chooseCRANmirror(ind=1)
# utils.install_packages('Rssa')

rssa = importr('Rssa')

In [ ]:
N = 700
w1 = 1/10
w2 = 1/5
C1 = 1
C2 = 2
phi1 = 0
phi2 = np.pi/2
Q = 301  # 301 номер, значит разладка в ряде будет на 302й точке, если ряд задан с 0.
B = 100
T_ = 100
L = 50
r = 2
noiseVariance = 0.5


iterationsNumber = 300

method = "svd"

In [ ]:
def plotSeries(s, title='Series', w=16, h=4):
    plt.figure(figsize=(w, h))
    plt.title(title)
    plt.plot(s)

Зададим наши функции

In [ ]:
seriesPermanent = lambda n: C1*np.sin(2*np.pi*w1*n + phi1) if n < Q-1 else C1*np.sin(2*np.pi*w2*n + phi1)
seriesTemporary = lambda n: C1*np.sin(2*np.pi*w1*n + phi1) if n < Q-1 else C2*np.sin(2*np.pi*w1*n + phi1)
seriesShifted = lambda n: C1*np.sin(2*np.pi*w1*n + phi1) if n < Q-1 else C1*np.sin(2*np.pi*w1*n + phi2)
seriesOutlier = lambda n: C1*np.sin(2*np.pi*w1*n + phi1)

Сгенерируем ряды с шумомом и без

In [ ]:
np.random.seed(0)
eps = np.random.normal(scale=noiseVariance, size=N)

fPerm = [seriesPermanent(i) for i in range(N)]
fPermNoise = fPerm + eps

fTemp = [seriesTemporary(i) for i in range(N)]
tmp = deepcopy(eps)
tmp[:Q] = tmp[:Q]/2
fTempNoise = fTemp + tmp

fShifted = [seriesShifted(i) for i in range(N)]
fShiftedNoise = fShifted + eps


fOutlier = [seriesOutlier(i) for i in range(N)]
fOutlier[Q] = fOutlier[Q] + C1*10
fOutlierNoise = fOutlier + eps

---

Посмотрим на графики строковых функций разладки в случае *постоянной* разладки при разных значениях частот ряда.

In [ ]:
def generate_series(omega, C1=1, C2=2, outlierVal=10, N=700, Q=301):
    w1, w2 = omega
    seriesPermanent = lambda n: C1*np.sin(2*np.pi*w1*n + phi1) if n < Q-1 else C1*np.sin(2*np.pi*w2*n + phi1)
    seriesTemporary = lambda n: C1*np.sin(2*np.pi*w1*n + phi1) if n < Q-1 else C2*np.sin(2*np.pi*w1*n + phi1)
    seriesShifted = lambda n: C1*np.sin(2*np.pi*w1*n + phi1) if n < Q-1 else C1*np.sin(2*np.pi*w1*n + phi2)
    seriesOutlier = lambda n: C1*np.sin(2*np.pi*w1*n + phi1)
    
    fPerm = [seriesPermanent(i) for i in range(N)]
    fTemp = [seriesTemporary(i) for i in range(N)]
    fShifted = [seriesShifted(i) for i in range(N)]
    fOutlier = [seriesOutlier(i) for i in range(N)]
    fOutlier[Q] = fOutlier[Q] + C1*outlierVal
    return dict(zip(['Permanent', 'Temporary', 'Shifted', 'Outlier'], [fPerm, fTemp, fShifted, fOutlier]))

def generate_hm_for_series(dictSeries, which):
    dictHM = {}
    for typeH, valH in zip(dictSeries.keys(), dictSeries.values()):
        if typeH not in which:
            continue
        dictHM[typeH] = Hmatr(valH, B, T_, L, neig=r, svdMethod=method)
    return dictHM

def get_det_func(dictHM, which):
    dictHMFunc = {}
    for typeHM, HM in zip(dictHM.keys(), dictHM.values()):
        dictFunc = {}
        for func in which:
            if func == 'row':
                dictFunc[func] = HM.getRow()
            if func == 'col':
                dictFunc[func] = HM.getCol()
            if func == 'sym':
                dictFunc[func] = HM.getSym()
            if func == 'diag':
                dictFunc[func] = HM.getDiag()
        dictHMFunc[typeHM] = dictFunc
    return dictHMFunc

def generate_funcs_for_diff_omegas(omegas, whichH, whichF):
    ans = {}
    for omega in omegas:
        ans[str(omega)] = get_det_func(generate_hm_for_series(generate_series(omega), whichH), whichF)
    return ans

def plot_heter_func(freqStats, which):
    
    len_types_hm = len(freqStats[list(freqStats.keys())[0]].keys())
    len_types_func = len(which)
    plt.figure(figsize=(10 * len_types_func, 6 * len_types_hm))
    for omegas, heterogeneity in zip(freqStats.keys(), freqStats.values()):
        for i, (typeH, funcs) in enumerate(zip(heterogeneity.keys(), heterogeneity.values())):
            for j, (typeF, vals) in enumerate(zip(funcs.keys(), funcs.values())):
                plt.subplot(len_types_hm, len_types_func, i * 4 + j + 1)
                plt.plot(vals, label = omegas)
                plt.title(f"{typeH} - {typeF}")
                plt.xlabel(r"$x_i$")
                plt.ylabel(r"$f(x_i)$")
                plt.ylim(0, 1)
                plt.legend()
    plt.show()

In [ ]:
which_heterogeneity = ['Permanent']
which_funcs = ['row']
omegas = ((1/10, 1/20), (1/10, 1/11), (1/10, 1/10.1))
plot_heter_func(generate_funcs_for_diff_omegas(omegas, which_heterogeneity, which_funcs), which_funcs)

Проверим еще раз.

In [ ]:
generate_hm_for_series(generate_series((1/10, 1/11)), which_heterogeneity)['Permanent'].plotHeterFunc()

In [ ]:
tmp = generate_hm_for_series(generate_series((1/10, 1/10.1)), which_heterogeneity)['Permanent'].plotHeterFunc()

In [ ]:
tmp = generate_hm_for_series(generate_series((1/10, 1/10.000001)), which_heterogeneity)
tmp['Permanent'].plotHeterFunc()

Из эмпирических наблюдений следует вывод:  
**Чем сильнее различия в рядах, тем лучше метод определяет наличие неоднородности**.


---

# Анализ переходного интервала (численный)

In [ ]:
import warnings
warnings.filterwarnings('ignore')

from utils.hmatrStatistics import Hmatr

In [ ]:
L=50
hm = Hmatr(fPerm, B, T_, L, neig=r, svdMethod=method)
data = hm.compute_distance(0)
print(np.sum(data[220], axis=1))
data[251]

In [ ]:
L=10
hm = Hmatr(fPerm, B, T_, L, neig=r, svdMethod=method)
data = hm.compute_distance(0)
print(np.sum(data[220], axis=1))
data[220]

In [ ]:
data[240]

In [ ]:
L=90
hm = Hmatr(fPerm, B, T_, L, neig=r, svdMethod=method)
data = hm.compute_distance(0)
print(np.sum(data[220], axis=1))
data[211]

In [ ]:
data[221]

In [ ]:
data[231]

In [ ]:
data[241]